In [117]:
from utils import *
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta

In [118]:
config = read_config('../config/main_config.yml')
print(config)
today_data = datetime.today().strftime('%Y-%m-%d')
today_data

{'inputs': '../inputs/', 'outputs': '../outputs/', 'selenium_reqs': {'do_scape': False, 'driver_path': '/Users/shubhamjuneja/vscode/personal_projects/webscraping/selenium/chromedriver-mac-x64 3/chromedriver', 'website_path': 'https://finance.yahoo.com', 'saving_outputs': '../outputs/output_from_webscrape/', 'downloaded_file_location': '/Users/shubhamjuneja/Downloads', 'op_path_n_name': '../outputs/output_from_webscrape/company_to_symbol_mapping.csv'}, 'yfinance_reqs': {'input_path': '../inputs/input_for_yfinance', 'op_path': '../outputs/output_from_yfinance', 'nse_file_name': '../outputs/output_from_webscrape/MW-NIFTY-50-16-Jun-2024.csv'}, 'modeling_reqs': {'m_input_path': '../inputs/input_for_modeling'}}


'2024-07-02'

In [119]:
mappings.loc[mappings['Symbols']=='BAJAJFINSV',]

,Symbols
28,BAJAJFINSV


In [120]:
mappings = pd.read_csv(config['yfinance_reqs']['nse_file_name'])
mappings.head()
final_list = []
for i in range(len(mappings)):
#for i in [28]:
    
    data = pd.read_csv(os.path.join(config['yfinance_reqs']['op_path'],mappings['Symbols'][i]+'.csv'))
    data['Date'] = pd.to_datetime(data['Date'])

    if len(data)>0:
        data['50_Moving_Avg'] = data['Close'].rolling(window=50).mean()
        data['200_Moving_Avg'] = data['Close'].rolling(window=200).mean()
        potential_buy=data.loc[data['Close']<data['50_Moving_Avg'],].sort_values(by='Date',ascending=False).reset_index(drop=True)

        req_date_id = data.loc[data['Date'].isin(potential_buy['Date']),].index-1
        potential_buy.columns = 'buying_date_'+potential_buy.columns 
        req_dates_data=data.iloc[req_date_id,].sort_values(by='Date',ascending=False).reset_index(drop=True)
        final_data = pd.concat([potential_buy,req_dates_data],axis = 1)
        rule_cleck = final_data.loc[(final_data['buying_date_Close']<final_data['buying_date_50_Moving_Avg']) &
                    (final_data['Close']>final_data['50_Moving_Avg']),][['buying_date_Date','Date','buying_date_Close','Close','buying_date_50_Moving_Avg','50_Moving_Avg']].reset_index(drop=True)
        rule_cleck_1 = rule_cleck.loc[rule_cleck['buying_date_Date']==today_data,].reset_index(drop=True)
        if len(rule_cleck_1)>0:
            final_list.append(mappings['Symbols'][i])


In [121]:
final_list

['LICI',
 'TITAN',
 'BANKBARODA',
 'CANBK',
 'NHPC',
 'SRF',
 'INDIANB',
 'SBICARD',
 'IDFCFIRSTB',
 'SUNDARMFIN',
 'FORTIS',
 'IDFC',
 'LAXMIMACH',
 'TTML',
 'JINDALSAW',
 'JUBLINGREA',
 'GOCOLORS',
 'GOPAL',
 'ZAGGLE',
 'JTLIND',
 'BFUTILITIE',
 'SHALBY',
 'CAPACITE',
 'RPTECH',
 'GEOJITFSL',
 'SMCGLOBAL',
 'JAYBARMARU',
 'ACLGATI',
 'CHEVIOT',
 'DMCC',
 'WEALTH',
 'RBZJEWEL',
 'OSWALAGRO',
 'SADBHAV',
 'SOUTHWEST',
 'LOYALTEX',
 'RVHL',
 'VARDMNPOLY',
 'ISFT',
 'UNIVASTU',
 'MITCON',
 'TOUCHWOOD',
 'VASWANI',
 'CELEBRITY',
 'LEXUS',
 'ONELIFECAP',
 'TECILCHEM',
 'CYBERMEDIA',
 'WILLAMAGOR',
 'FMNL']